In [17]:
import pandas as pd
import urllib.request
import zipfile

In [18]:
url="https://github.com/mattharrison/datasets/raw/master/data/kaggle-survey-2018.zip"
fname='kaggle-survey-2018.zip'
member_name='multipleChoiceResponses.csv'


In [19]:
def extract_zip(src,dest, member_name):
    url=src
    fname=dest
    fin=urllib.request.urlopen(url)
    data=fin.read()

    with open(dest,mode='wb') as fout:
        fout.write(data)
    with zipfile.ZipFile(dest) as z:
        kag=pd.read_csv(z.open(member_name))
        kag_question=kag.iloc[0]
        raw=kag.iloc[1:]
        return raw

In [20]:
raw = extract_zip(url,fname,member_name)

C:\Users\Tyreek ALEXANDER\AppData\Local\Temp\ipykernel_948\2146811015.py:10: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types. Specify dtype option on import or set low_memory=False.
  kag=pd.read_csv(z.open(member_name))
